In this notebook, we will calculate features using Trotterization with a noise-free classical simulator (AerSimulator).

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# This is necessary only when running the notebook in the docker container
%cd fourier_learning_ibm/

/root/workdir/fourier_learning_ibm


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
# GPU check
!nvidia-smi -L

GPU 0: NVIDIA A100-PCIE-40GB (UUID: GPU-3fac7503-bb55-35cd-f566-c919eaa55798)
GPU 1: NVIDIA A100-PCIE-40GB (UUID: GPU-bf03abcd-ef36-6733-da57-460c8eb9cab4)
GPU 2: NVIDIA A100-PCIE-40GB (UUID: GPU-bc01f8f3-0939-f2c0-6998-dd49d0360463)
GPU 3: NVIDIA A100-PCIE-40GB (UUID: GPU-b7aeac00-b332-1045-eebc-bb5653ac8402)


In [4]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import os
from heisenberg import (
    HeisenbergModelGPU,
    get_graph,
    get_positions,
    # get_initial_layout,
    # get_prob0,
)
from setup import setup_backend
from helper_features import run_job, get_features
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, Batch
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
import pickle
import networkx as nx
from datetime import datetime, timezone
import json
import math

In [5]:
import warnings

warnings.filterwarnings("ignore")

### Config backend 

In [6]:
# density matrix simulator
backend_qpu, backend_dm_noiseless, backend_dm_noisy = setup_backend(
    qpu_name="ibm_marrakesh", device="GPU", method="density_matrix"
)

# matrix product state simulator
backend_qpu, backend_mps_noiseless, backend_mps_noisy = setup_backend(
    qpu_name="ibm_marrakesh", device="GPU", method="matrix_product_state"
)

Using backend QPU: <IBMBackend('ibm_marrakesh')>

NoiseModel:
  Basis gates: ['cz', 'delay', 'for_loop', 'id', 'if_else', 'measure', 'reset', 'rz', 'switch_case', 'sx', 'x']
  Instructions with noise: ['sx', 'x', 'reset', 'id', 'measure', 'cz']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155]
  Specific qubit errors: [('x', (0,)), ('x', (1,)), ('x', (2,))

### Prepare parameters and load dataset

In [7]:
n_qubits = 32

C = 3
n_samples = 55
n_shots = 1000

if n_qubits <= 20:  # n_qubits <= 20
    threshold = 0.6

    # Load dataset (orient="records")
    if n_qubits == 20:  # For 20 qubits, use the 12-qubit dataset
        df_n_step = pd.read_json(
            f"results/get_nstep/12Q/threshold{threshold:.2f}/optimal_n_step.json",
            orient="records",
        )
    else:
        df_n_step = pd.read_json(
            f"results/get_nstep/{n_qubits}Q/threshold{threshold:.2f}/optimal_n_step.json",
            orient="records",
        )
    display(df_n_step)

    # Note that K = n_features-1.
    # n_features counts only the number of real features, also the one of t=0, i.e., constant term.
    # The number of all features, including imaginary numbers, is 2K+1.
    n_features = len(df_n_step)

    times = df_n_step["t"].values
    n_step_array = df_n_step["n_step"].values

else:  # n_qubits > 20
    threshold = None
    n_features = 12
    # Generate n_features points at intervals of np.pi/C
    times = np.linspace(0, np.pi / C * (n_features - 1), n_features)

    # If n_qubits > 20, n_step takes integer values from 1 to max_n_step.
    if n_qubits == 32:
        max_n_step = 4
    elif n_qubits == 40:
        max_n_step = 5
    # Generate n_features equally spaced numbers from 1 to max_n_step
    n_step_array = np.linspace(1, max_n_step, n_features)
    # Round each element to the nearest integer
    n_step_array = [round(x) for x in n_step_array]

print(f"n_qubits: {n_qubits}")
print(f"C: {C}")
print(f"threshold: {threshold}")
print(f"n_samples: {n_samples}")
print(f"n_shots: {n_shots}")
print(f"n_features: {n_features}")
print(f"times: {times}")
print(f"n_step_array: {n_step_array}")

# Load dataset (orient="records")
df_dataset = pd.read_json(
    f"results/get_dataset/{n_qubits}Q/dataset.json",
    orient="records",
)
all_Js = df_dataset["Js"].values
all_expected_values = df_dataset["expected_value"].values
n_samples = len(all_Js)
display(df_dataset)

n_qubits: 32
C: 3
threshold: None
n_samples: 55
n_shots: 1000
n_features: 12
times: [ 0.          1.04719755  2.0943951   3.14159265  4.1887902   5.23598776
  6.28318531  7.33038286  8.37758041  9.42477796 10.47197551 11.51917306]
n_step_array: [1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4]


,sample,Js,expected_value,expected_value_diag,diff
0,0,"[0.0335840776, -0.0074928487000000005, 0.04396...",0.929046,NaN,NaN
1,1,"[0.0665962073, -0.0248109713, -0.0184528614, -...",1.087532,NaN,NaN
2,2,"[0.009791546600000001, 0.039774732, 0.02022040...",1.179907,NaN,NaN
3,3,"[-0.0160006353, -0.026693271100000002, 0.01752...",1.167376,NaN,NaN
4,4,"[-0.063179355, -0.0008282459, -0.0234502427000...",1.456877,NaN,NaN
5,5,"[0.0206089153, -0.0261758708, 0.0190752775, 0....",0.809953,NaN,NaN
6,6,"[-0.014775294900000001, -0.047425730900000004,...",1.314305,NaN,NaN
7,7,"[-0.0473192182, 0.0537731035, -0.012881246, -0...",0.911797,NaN,NaN
8,8,"[-0.0151089676, -0.0089006608, 0.01824507, 0.0...",1.444066,NaN,NaN
9,9,"[0.042361312500000005, -0.0550761996, -0.03842...",0.946791,NaN,NaN


# Calculate Fourier feature

## Trotter simulation (Simulator, Noise free)

In [8]:
config = {
    "n_qubits": n_qubits,
    "n_samples": n_samples,
    "n_shots": n_shots,
    "n_features": n_features,
    "times": times,
    "all_Js": all_Js,
    "n_step_array": n_step_array,
    "backend": backend_mps_noiseless,  # This is important!!
    "all_expected_values": all_expected_values,
}

In [10]:
# For simulator, we need job instances instead of job ids
# progress is saved to results/fourier_feature_sim_noisy/temp_progress.txt
_, jobs, _ = run_job(config, backend_qpu, sim_type="noiseless")

In [11]:
%%time
features_df = get_features(config, jobs)
display(features_df)

Post-processing sample 0/55
Post-processing sample 1/55
Post-processing sample 2/55
Post-processing sample 3/55
Post-processing sample 4/55
Post-processing sample 5/55
Post-processing sample 6/55
Post-processing sample 7/55
Post-processing sample 8/55
Post-processing sample 9/55
Post-processing sample 10/55
Post-processing sample 11/55
Post-processing sample 12/55
Post-processing sample 13/55
Post-processing sample 14/55
Post-processing sample 15/55
Post-processing sample 16/55
Post-processing sample 17/55
Post-processing sample 18/55
Post-processing sample 19/55
Post-processing sample 20/55
Post-processing sample 21/55
Post-processing sample 22/55
Post-processing sample 23/55
Post-processing sample 24/55
Post-processing sample 25/55
Post-processing sample 26/55
Post-processing sample 27/55
Post-processing sample 28/55
Post-processing sample 29/55
Post-processing sample 30/55
Post-processing sample 31/55
Post-processing sample 32/55
Post-processing sample 33/55
Post-processing sample 3

,sample_id,f_0 Re,f_1 Re,f_1 Im,f_2 Re,f_2 Im,f_3 Re,f_3 Im,f_4 Re,f_4 Im,...,f_7 Im,f_8 Re,f_8 Im,f_9 Re,f_9 Im,f_10 Re,f_10 Im,f_11 Re,f_11 Im,expected_value
0,0,1.0,0.983024,-0.073388,0.953101,-0.179845,0.890121,-0.250400,0.863900,-0.317442,...,-0.474048,0.461913,-0.515254,0.350305,-0.517973,0.289111,-0.525357,0.201212,-0.487799,0.929046
1,1,1.0,0.990750,0.092414,0.955021,0.191507,0.905062,0.272295,0.885226,0.259400,...,0.394342,0.603859,0.392285,0.540510,0.400921,0.456458,0.354750,0.387287,0.309103,1.087532
2,2,1.0,0.978088,0.161075,0.915391,0.332440,0.822490,0.453002,0.679813,0.570757,...,0.640897,0.064530,0.602217,-0.059473,0.537811,-0.148767,0.421082,-0.257809,0.337959,1.179907
3,3,1.0,0.984462,0.109409,0.918445,0.243976,0.839764,0.389625,0.766438,0.469048,...,0.528248,0.366594,0.558088,0.305104,0.535590,0.296296,0.484539,0.255410,0.472531,1.167376
4,4,1.0,0.928702,0.348829,0.692346,0.692383,0.379447,0.855942,0.003430,0.871055,...,0.241888,-0.587811,-0.073531,-0.420028,-0.265625,-0.215895,-0.400433,-0.016542,-0.358100,1.456877
5,5,1.0,0.965068,-0.222944,0.856743,-0.463753,0.714208,-0.611327,0.531563,-0.723928,...,-0.740829,-0.258660,-0.667304,-0.415292,-0.475629,-0.497685,-0.350898,-0.540066,-0.223536,0.809953
6,6,1.0,0.960937,0.257625,0.828666,0.525297,0.658526,0.703330,0.383657,0.855858,...,0.797094,-0.560525,0.596353,-0.731874,0.382430,-0.784421,0.136074,-0.754693,-0.110670,1.314305
7,7,1.0,0.987939,-0.111414,0.949825,-0.226514,0.910061,-0.320010,0.874816,-0.346475,...,-0.524777,0.501245,-0.547762,0.431866,-0.549178,0.293635,-0.543768,0.199632,-0.520642,0.911797
8,8,1.0,0.928653,0.353481,0.683530,0.705062,0.373284,0.874140,0.013332,0.920773,...,0.299911,-0.782370,0.052778,-0.705440,-0.262928,-0.487521,-0.491948,-0.217927,-0.611612,1.444066
9,9,1.0,0.989247,-0.082125,0.973741,-0.149975,0.912246,-0.203551,0.851012,-0.178144,...,-0.226275,0.554794,-0.244893,0.487481,-0.233555,0.426830,-0.151049,0.349234,-0.115125,0.946791


CPU times: user 5h 8min 22s, sys: 1h 30min 41s, total: 6h 39min 4s
Wall time: 8min 2s
